# PyVista External Windows in Jupyter (Updated)

This notebook shows how to create PyVista plots in external windows from Jupyter notebooks using current PyVista versions.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pyvista as pv
import complexplorer as cp
from complexplorer.cmap import Phase

## Understanding PyVista Backends

In newer PyVista versions, the available Jupyter backends are:
- **'static'** - Static images inline
- **'client'** - Client-side rendering
- **'server'** - Server-side rendering
- **'trame'** - Interactive inline (default)
- **'html'** - HTML export
- **'none'** - Disable Jupyter integration

The 'qt' backend is no longer available for Jupyter.

In [ ]:
# Check available backends
print(f"Current backend: {pv.global_theme.jupyter_backend}")
print(f"PyVista version: {pv.__version__}")

## Method 1: Force External Window with notebook=False

The most reliable way to get external windows is to create plotters with `notebook=False`:

In [ ]:
# Create a plotter that will use external window
plotter = pv.Plotter(notebook=False)

# Add a test sphere
sphere = pv.Sphere()
plotter.add_mesh(sphere, color='cyan', smooth_shading=True)
plotter.add_text("External Window Test", font_size=20)

# This will open in a separate window
plotter.show()

## Method 2: Modify complexplorer Functions

Since complexplorer functions don't expose the notebook parameter, we need a workaround:

In [ ]:
# Temporarily set backend to 'none' to disable inline display
original_backend = pv.global_theme.jupyter_backend
pv.set_jupyter_backend('none')

# Now plots will try to open externally
f = lambda z: (z - 1) / (z + 1)

try:
    # This should open in external window
    cp.riemann_pv(
        f,
        scaling='arctan',
        cmap=Phase(12),
        title="Möbius transformation - External Window",
        n_theta=150,
        n_phi=150,
        interactive=True
    )
except Exception as e:
    print(f"Error: {e}")
finally:
    # Restore original backend
    pv.set_jupyter_backend(original_backend)

## Method 3: Create a Wrapper Function

A better approach is to create wrapper functions that force external windows:

In [ ]:
def riemann_external(func, **kwargs):
    """Wrapper to show riemann_pv in external window."""
    # Save current backend
    original_backend = pv.global_theme.jupyter_backend
    
    try:
        # Disable Jupyter integration
        pv.set_jupyter_backend('none')
        
        # Call the function
        return cp.riemann_pv(func, **kwargs)
    finally:
        # Restore backend
        pv.set_jupyter_backend(original_backend)

# Use the wrapper
f = lambda z: z**2 - 1
riemann_external(f, scaling='arctan', cmap=Phase(12), 
                 title="z² - 1 (External Window)", n_theta=200, n_phi=200)

## Method 4: Direct PyVista Approach

For full control, create the visualization directly with PyVista:

In [ ]:
from complexplorer.mesh_utils import RectangularSphereGenerator, stereographic_projection, ModulusScaling

# Generate sphere mesh
n_theta, n_phi = 100, 100
generator = RectangularSphereGenerator(radius=1.0, n_theta=n_theta, n_phi=n_phi)
sphere = generator.generate()

# Apply stereographic projection and function
points = sphere.points
x, y, z = points[:, 0], points[:, 1], points[:, 2]
w = stereographic_projection(x, y, z, from_north=True)

# Evaluate function
f = lambda z: (z - 1) / (z + 1)
f_vals = f(w)

# Get colors
cmap = Phase(12)
rgb = cmap.rgb(f_vals.reshape(-1, 1)).squeeze()
sphere["RGB"] = rgb

# Apply modulus scaling
moduli = np.abs(f_vals)
radii = ModulusScaling.arctan(moduli, 0.2, 1.0)
scaled_points = points * radii[:, np.newaxis]
sphere.points = scaled_points

# Create external window plotter
plotter = pv.Plotter(notebook=False, window_size=(1000, 1000))
plotter.add_mesh(sphere, rgb=True, smooth_shading=True)
plotter.add_text("Direct PyVista - External Window", font_size=16)
plotter.show()

## Method 5: Running as a Script

Another option is to save your visualization code as a Python script and run it:

In [ ]:
# Save visualization code to a file
script_content = '''
import complexplorer as cp
from complexplorer.cmap import Phase

# Define function
f = lambda z: (z**2 - 1) / (z**2 + 1)

# Create high-quality visualization
cp.riemann_pv(
    f,
    scaling='arctan',
    cmap=Phase(12),
    title="High Quality External Visualization",
    n_theta=200,
    n_phi=200,
    high_quality=True,
    anti_aliasing=True
)
'''

with open('temp_viz.py', 'w') as f:
    f.write(script_content)

print("Script saved as temp_viz.py")
print("Run it from terminal: python temp_viz.py")

In [ ]:
# Or run it directly from the notebook
import subprocess
subprocess.Popen(['python', 'temp_viz.py'])
print("External window should open...")

## Comparison: Inline vs External

Let's compare the quality difference:

In [ ]:
# First show inline with trame (lower quality)
pv.set_jupyter_backend('trame')

f = lambda z: z**3 - 1
print("Inline display (lower quality):")
cp.riemann_pv(f, scaling='arctan', cmap=Phase(12), 
              title="z³ - 1 (Inline)", n_theta=100, n_phi=100)

In [ ]:
# Now external (high quality)
print("\nFor comparison, run this for external window:")
print("riemann_external(f, scaling='arctan', cmap=Phase(12), title='z³ - 1 (External)', n_theta=200, n_phi=200)")

## Best Practices

1. **For publication quality**: Always use external windows
2. **For quick exploration**: Use inline display
3. **For presentations**: Save high-res images from external windows
4. **For documentation**: Use static backend for consistent appearance

### Quality differences:
- **External windows**: Full anti-aliasing, better shading, higher resolution
- **Inline (trame)**: Compressed, limited resolution, some aliasing
- **Static**: No interaction but consistent appearance

In [ ]:
# Clean up
import os
if os.path.exists('temp_viz.py'):
    os.remove('temp_viz.py')
    print("Cleaned up temporary files")